In [1]:
import functools
import random
import math
import pandas as pd
import numpy as np
from sklearn import svm
from sklearn.svm import SVR
from sklearn.pipeline import Pipeline

In [2]:
# Variables

# File prefix
file_pre = 'normw_data_'

# File suffix
file_suf = '.csv'

# Start year
start = 1990

# Number of years to process
end = 21 

# Number of iterations
runs = 5

# Training/Test percent split
training_percent = 0.85

In [3]:
# Create file list
all_files = []
for i in range(end+1):
    all_files.append(file_pre + str(start + i) + file_suf)

random.shuffle(all_files)

# Need data for if hit or not
hits = pd.read_csv('labelled_data.csv', encoding = "ISO-8859-1")
hits['song'] = hits['title'].str.lower() 
hits['artist_name'] = hits['artist_name'].str.lower() 

# Functions for Metrics
def acc(data):
    in_ac = [float(1) if a==b else 0 for (a,b) in data]
    return functools.reduce(lambda x, y: x + y, in_ac) / len(in_ac)
def pr(data):
    l = len(data)
    tp = functools.reduce(lambda x, y: x + y, [float(1) if a==b else 0 for (a,b) in data])
    fp = functools.reduce(lambda x, y: x + y, [float(1) if a==0 and p==1 else 0 for (a,p) in data])
    fn = functools.reduce(lambda x, y: x + y, [float(1) if a==1 and p==0 else 0 for (a,p) in data])
    
    pr = tp / (tp + fp)
    rc = tp / (tp + fn)
    return (pr, rc)

In [4]:
# Ingest files, randomize order, train model, predict for test set, and score.

for i in range(runs):
    print ("Run #" + str(i))
    
    # Ingest and randomize row order
    random.shuffle(all_files)
    df = pd.DataFrame()
    for file in all_files:
        chunks = pd.read_csv(file, encoding = "ISO-8859-1", chunksize=1000, low_memory=False,
                           usecols=['artist_name', 'song', 'artist_nid','song_nid', 'bars', 'beats',
                                    'sections_start', 'segments', 'segments_loudness_max', 'tatums'])
        tmp = pd.concat((chunk for chunk in chunks), ignore_index=True)
        df = pd.concat([df, tmp], ignore_index=True)

    df = pd.merge(df, hits, on=['song', 'artist_name'], how='inner').reindex(np.random.permutation(df.index))
    
    # Split training/testing dataset
    train_num = math.ceil (training_percent * len(df.index))
    test_num = len(df.index) - train_num
    train_df = df.head(train_num)
    test_df = df.tail(test_num)
    
    # Train Model
    # Set max_iter=100 to speed up processing and model convergence    
    clf = svm.SVC(kernel='rbf', C=1e3, gamma=0.001, cache_size=6000, max_iter=100)
    y = train_df["hit"].values
    X = train_df[['bars', 'beats', 'sections_start', 'segments', 'segments_loudness_max', 'tatums']].values
    clf.fit(X, y)

    # Predict and score
    sample_size = 0 # Use whole test data set
    sample = test_df
    if sample_size:
        sample = sample.sample(sample_size)
    results = []
    for index, row in sample.iterrows():
        tmp_df = pd.DataFrame([row])
        pred = clf.predict(tmp_df[['bars', 'beats', 'sections_start', 'segments', 'segments_loudness_max', 'tatums']].values)
        results.append((row['hit'], pred[0]))
    print ("acc: " + str(acc(results)))
    print ("precision/recall:" + str(pr(results)))
    
    

Run #0


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\base.py:220: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


acc: 0.8835892852306275
precision/recall:(0.9068998109640832, 0.9717323960443226)
Run #1
acc: 0.8422036239538474
precision/recall:(0.8637979887771543, 0.9711724654881629)
Run #2
acc: 0.7018498957233011
precision/recall:(0.714014107792351, 0.9763017104965714)
Run #3
acc: 0.8429078302321172
precision/recall:(0.8672909177047627, 0.9677228769551292)
Run #4
acc: 0.7438314238509249
precision/recall:(0.7632851584213451, 0.966870863258696)
